In [20]:
import numpy as np
from time import time
from functools import wraps

In [21]:
def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print(f'func:{f.__name__} took: {te-ts:.4f} sec')
        return result
    return wrap

### Binomial Tree Representation

Stock tree can be represented using nodes $(i,j)$ and inital stock price $ S_{0} $

$$ S_{i,j} = S_{0}u^{j}d^{i-j}

We are pricing a European Call

$$ C_{ N, j} = max(S_{ N, j} - K, 0) $$

In [33]:
s0 = 100
k = 150
T = 1
r = 0.06
N = 1000 
u = 1.1
d = 1/u  # we want to ensure it is recombining tree 
optype = 'C'  # 'C' for Call, 'P' for Put

### Bionmial Tree Slow

In [22]:
@timing

def binomial_tree_slow(k,T,s0,u,d,N,optype ="C"):
    # precompute constants
    
    dt = T / N # length of time interval
    
    q = (np.exp(r * dt) - d)/(u - d) # the risk-neutral probability parameter
    
    disc = np.exp(-r * dt) # discount factor per time step

    # initialize asset prices at maturity - Time step N

    S = np.zeros(N + 1)
    S[0] = s0 * d**N
 
    for j in range(1, N + 1):
        S[j] = S[j - 1] * u / d
    
    # initialize option values at maturity

    C = np.zeros(N + 1)
    for j in range(N+1):
        C[j] = max(0, S[j] - k)
    
    # step backwards through tree

    for i in np.arange(N, 0, -1): 
        for j in range(0, i):
            C[j] = disc * (q * C[j + 1] + (1 - q) * C[j])
    
    return C[0]

In [23]:
print(binomial_tree_slow(k,T,s0,u,d,N,optype ="C"))

func:binomial_tree_slow took: 0.0000 sec
10.145735799928817


### Fast Tree

In [26]:
@timing

def binomial_tree_fast(k,T,s0,u,d,N,optype ="C"):
    # precompute constants
    
    dt = T / N # length of time interval
    
    q = (np.exp(r * dt) - d)/(u - d) # the risk-neutral probability parameter
    
    disc = np.exp(-r * dt) # discount factor per time step

    # initialize asset prices at maturity - Time step N
    C = s0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1))
    # initialize option values at maturity
    C = np.maximum(C - k , np.zeros(N + 1))
    
    # step backwards through tree
    for i in np.arange(N, 0, -1): 
        C = disc * ( q * C[1:i+1] + (1 - q) * C[0:i] )
    
    return C[0]

In [ ]:
print(binomial_tree_fast(k,T,s0,u,d,N,optype ="C"))

func:binomial_tree_fast took: 0.0030 sec
10.145735799928826


In [34]:
for N in [100,1000,5000]:
    print(binomial_tree_slow(k,T,s0,u,d,N,optype ="C"))
    print(binomial_tree_fast(k,T,s0,u,d,N,optype ="C"))

func:binomial_tree_slow took: 0.0020 sec
26.158393128807546
func:binomial_tree_fast took: 0.0000 sec
26.158393128807518
func:binomial_tree_slow took: 0.1921 sec
84.39093697201648
func:binomial_tree_fast took: 0.0040 sec
84.3909369720167
func:binomial_tree_slow took: 4.7883 sec
99.91045880152151
func:binomial_tree_fast took: 0.0339 sec
99.91045880152109
